<a href="https://colab.research.google.com/github/Rajatvs/ML-2/blob/main/FOIL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries
!pip install sklearn ripper

# Importing necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import random

# 1. Load a dataset
# For demonstration purposes, we'll use the Iris dataset available from sklearn
iris = load_iris()
X = iris.data
y = iris.target

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 2. Encode labels (if necessary)
# For multi-class classification, we might need to encode labels (but Iris is already numeric)
encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train)
y_test_encoded = encoder.transform(y_test)

# 3. Example-based method: Using the RIPPER (Repeated Incremental Pruning to Produce Error Reduction) algorithm
from ripper import RIPPER

# Create and train the RIPPER model on the training data
ripper_model = RIPPER()
ripper_model.fit(X_train, y_train_encoded)

# Evaluate the RIPPER model on the test set
y_pred_ripper = ripper_model.predict(X_test)
ripper_accuracy = accuracy_score(y_test_encoded, y_pred_ripper)
print(f"RIPPER Accuracy: {ripper_accuracy * 100:.2f}%")

# 4. FOIL (First-Order Inductive Learner) for learning first-order rules

# FOIL Algorithm: Simplified example
class FOIL:
    def __init__(self, max_depth=3):
        self.max_depth = max_depth
        self.rules = []

    def fit(self, X_train, y_train):
        self.rules = []
        for target_class in set(y_train):
            self._learn_class(X_train, y_train, target_class)

    def _learn_class(self, X_train, y_train, target_class):
        # Basic FOIL rule learning: Just an example, not an optimized version
        for depth in range(self.max_depth):
            rule = self._generate_rule(X_train, y_train, target_class)
            if rule:
                self.rules.append(rule)
                # "Prune" dataset by removing examples that match the rule
                mask = [y == target_class for y in y_train]
                X_train = X_train[mask]
                y_train = y_train[mask]

    def _generate_rule(self, X_train, y_train, target_class):
        # Simple rule generation: Select a random feature to build a rule
        feature_index = random.choice(range(X_train.shape[1]))
        feature_value = X_train[random.randint(0, len(X_train) - 1)][feature_index]
        rule = f"IF feature_{feature_index} <= {feature_value} THEN class = {target_class}"
        return rule

    def predict(self, X_test):
        # Simplified prediction using the learned rules
        predictions = []
        for x in X_test:
            predicted_class = None
            for rule in self.rules:
                feature_index = int(rule.split(' ')[1][8:])
                if x[feature_index] <= float(rule.split(' ')[5]):
                    predicted_class = int(rule.split(' ')[8])
                    break
            predictions.append(predicted_class if predicted_class is not None else random.choice(range(len(set(y_train)))))
        return predictions

# 5. Apply FOIL
foil_model = FOIL()
foil_model.fit(X_train, y_train_encoded)

# Evaluate FOIL model
y_pred_foil = foil_model.predict(X_test)
foil_accuracy = accuracy_score(y_test_encoded, y_pred_foil)
print(f"FOIL Accuracy: {foil_accuracy * 100:.2f}%")

# 6. Display rules learned by FOIL
print("\nLearned FOIL Rules:")
for rule in foil_model.rules:
    print(rule)

# 7. Compare both models (RIPPER vs FOIL)
print(f"\nRIPPER Model Accuracy: {ripper_accuracy * 100:.2f}%")
print(f"FOIL Model Accuracy: {foil_accuracy * 100:.2f}%")
